# GDSC Raw Gene Expression Data Importation
**Local Version**: 2
**Source Version**: 6.0

This notebook will import raw GDSC expression data through the [GDSC](http://www.cancerrxgene.org/downloads) portal.  Normally, files for this are hosted on the [Sanger FTP Server](ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/) (release-6.0 in this case), but that is not true for this particular dataset.  Data for this comes from the cancerrxgene.org domain and in order to get cell line ids for this, it has to be joined to a file on the FTP server (see below)

Note that the GDSC exposes 2 expression datasets, labeled as the following:

1. Raw - "Expression array data for Cell lines"
2. Preprocessed - "RMA normalised expression data for Cell lines"

In this case option 2 will be used and the raw data for this is located [here](http://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip).  This file contains no cell line ids though so it has to be joined to [Cell_Lines_Details.xlsx](ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/release-6.0/Cell_Lines_Details.xlsx) on the FTP server.

In [2]:
%run -m ipy_startup
%run -m ipy_logging
%run -m ipy_seaborn
from mgds.data_aggregation import database as db
from mgds.data_aggregation import source as src
from mgds.data_aggregation import io_utils
import io
from py_utils import zip_utils
from py_utils.collection_utils import subset
pd.set_option('display.max_info_rows', 50000000)

## Load Raw Expression CSV

In [3]:
# Download Cell Line Details spreadsheet
url = 'http://www.cancerrxgene.org/gdsc1000/GDSC1000_WebResources/Data/preprocessed/Cell_line_RMA_proc_basalExp.txt.zip'
filepath = db.raw_file(src.GDSC_v2, 'gene-expression.zip')
filepath = io_utils.download(url, filepath, check_exists=True)
filepath

2016-11-21 13:44:17,618:DEBUG:mgds.data_aggregation.io_utils: Returning previously downloaded path for "/Users/eczech/data/research/mgds/raw/gdsc_v2_gene-expression.zip"


'/Users/eczech/data/research/mgds/raw/gdsc_v2_gene-expression.zip'

In [12]:
filenames = zip_utils.get_zip_archive_files(filepath)        
assert len(filenames) == 1, 'Files in zip archive do not match expected.  Files found: {}'.format(file_names)
assert filenames[0] == 'Cell_line_RMA_proc_basalExp.txt', \
    'Files in zip archive do not match expected.  Files found: {}'.format(file_names)

# Fetch raw bytes for csv file
d = zip_utils.get_zip_archive_file_data(filepath, filenames[0])

# Convert bytes to UTF8 string and parse as DataFrame
d = pd.read_csv(io.StringIO(d.decode('utf-8')), sep='\t')
d.head()

,GENE_SYMBOLS,GENE_title,DATA.906826,DATA.687983,DATA.910927,DATA.1240138,DATA.1240139,DATA.906792,DATA.910688,DATA.1240135,...,DATA.753584,DATA.907044,DATA.998184,DATA.908145,DATA.1659787,DATA.1298157,DATA.1480372,DATA.1298533,DATA.930299,DATA.905954.1
0,TSPAN6,tetraspanin 6 [Source:HGNC Symbol;Acc:11858],7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
1,TNMD,tenomodulin [Source:HGNC Symbol;Acc:17757],2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
2,DPM1,dolichyl-phosphate mannosyltransferase polypep...,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
3,SCYL3,SCY1-like 3 (S. cerevisiae) [Source:HGNC Symbo...,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
4,C1orf112,chromosome 1 open reading frame 112 [Source:HG...,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795


In [13]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17737 entries, 0 to 17736
Columns: 1020 entries, GENE_SYMBOLS to DATA.905954.1
dtypes: float64(1018), object(2)
memory usage: 138.0+ MB


In [5]:
# Show example rows with null Gene IDs
# pd.set_option('display.max_colwidth', 1000)
# d[d['GENE_SYMBOLS'].isnull()].iloc[:, :10]

In [6]:
# At TOW, the only fields with any null values were the two-non data columns.
# However, this is problematic since these identifiers are crucial and since they
# are not present the best thing to do seems to be to just remove any rows with
# no gene id

# First though, make sure there are no null values in other fields
c_null = d.columns[d.isnull().sum(axis=0) > 0].tolist()
assert c_null == ['GENE_SYMBOLS', 'GENE_title']

# And ensure that there are only two non-"DATA" columns
c_data = d.filter(regex='DATA.').columns.tolist()
assert len(d[[c for c in d if c not in c_data]].columns.tolist()) == 2

# Drop gene title now -- don't need this anymore
assert 'GENE_title' in d
d = d.drop('GENE_title', axis=1)

# Print the distribution of null gene ids
print(d['GENE_SYMBOLS'].isnull().value_counts().rename('Null Gene ID Counts'))

# Remove rows for absent GENE ids
d = subset(d, lambda df: df[df['GENE_SYMBOLS'].notnull()], subset_op='Remove rows with null gene ID')

# Rename gene id field
d = d.rename(columns={'GENE_SYMBOLS': 'GENE_ID:HGNC'})

# Make sure everything is now non-null
assert np.all(d.notnull())

False    17419
True       318
Name: Null Gene ID Counts, dtype: int64
[Remove rows with null gene ID] Records before = 17737, Records after = 17419, Records removed = 318 (%1.79)


In [7]:
# Convert cell line ID strings to proper COSMIC IDs

# First extract all column names corresponding to cell lines
c_cl = d.filter(regex='DATA.').columns.tolist()

# Convert these ids to numeric after stripping out string parts
n_cl = pd.to_numeric([c.replace('DATA.', '') for c in c_cl])

# Idenfity ids that end with a decimal of some kind (ie that end with ".1" instead of no decimal)
mask_dupe = n_cl != n_cl.astype(np.int64)

n_cl[mask_dupe]

array([ 1503362.1,  1330983.1,   909976.1,   905954.1])

In [8]:
# Where the above ids end with a decimal, assume these decimals indicate a second experiment was
# run for the same cell line with an ID equal to the integer part, and that these values are
# preferable over the ".0" instance (which should be ignored)
c_rm = ['DATA.{}'.format(int(v)) for v in n_cl[mask_dupe]]

logger.info(
    'Dropping the following cell line columns as they seem to have second runs '\
    'for the same experiment (indicated by columns with the same ID + ".1"): {}'\
    .format(c_rm)
)

n_col = d.shape[1]
d = d.drop(c_rm, axis=1)
assert n_col == d.shape[1] + len(c_rm)

# Clean up cell line ids to have names consisting of nothing (hopefully)
# but integer values as strings.  If any exceptions to this rule do not 
# end in ".1", then they will not be converted here but will cause a later
# error when converting these strings to integers (which is the desired behavior)
d = d.rename(columns=lambda c: c.replace('DATA.', '').replace('.1', ''))

d.head()

2016-11-21 11:40:14,774:INFO:root: Dropping the following cell line columns as they seem to have second runs for the same experiment (indicated by columns with the same ID + ".1"): ['DATA.1503362', 'DATA.1330983', 'DATA.909976', 'DATA.905954']


,GENE_ID:HGNC,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954
0,TSPAN6,7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,5.943046,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
1,TNMD,2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,2.686874,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
2,DPM1,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,10.299757,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
3,SCYL3,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,3.848112,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
4,C1orf112,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,3.352835,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795


## Melt to Long Format

In [9]:
# Melt to long format with gene ids preserved
d = pd.melt(d, id_vars='GENE_ID:HGNC', var_name='CELL_LINE_ID:COSMIC', value_name='VALUE')

# Convert COSMIC cell line IDs to integers and then to strings (this ensures that they can
# all be converted to integer first -- an error will be thrown otherwise)
d['CELL_LINE_ID:COSMIC'] = d['CELL_LINE_ID:COSMIC'].astype(np.int64).astype(str)

# Ensure that after melting and converting ids that there are no instances of multiple
# records for any one cell line + gene
assert not np.any(d[['CELL_LINE_ID:COSMIC', 'GENE_ID:HGNC']].duplicated()), \
    'Found at least one cell line + gene combination with multiple records'

d.head()

,GENE_ID:HGNC,CELL_LINE_ID:COSMIC,VALUE
0,TSPAN6,906826,7.632023
1,TNMD,906826,2.964585
2,DPM1,906826,10.379553
3,SCYL3,906826,3.614794
4,C1orf112,906826,3.380681


In [11]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17662866 entries, 0 to 17662865
Data columns (total 3 columns):
GENE_ID:HGNC           17662866 non-null object
CELL_LINE_ID:COSMIC    17662866 non-null object
VALUE                  17662866 non-null float64
dtypes: float64(1), object(2)
memory usage: 404.3+ MB


## Merge to Common Cell Line ID

In [12]:
d_meta = db.load(src.GDSC_v2, db.IMPORT, 'cellline-meta')[['CELL_LINE_ID', 'CELL_LINE_ID:COSMIC']]
d_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1034 entries, 0 to 1033
Data columns (total 2 columns):
CELL_LINE_ID           1034 non-null object
CELL_LINE_ID:COSMIC    1034 non-null object
dtypes: object(2)
memory usage: 24.2+ KB


In [13]:
d_exp = pd.merge(d, d_meta, on='CELL_LINE_ID:COSMIC', how='left')
d_exp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17749961 entries, 0 to 17749960
Data columns (total 4 columns):
GENE_ID:HGNC           17749961 non-null object
CELL_LINE_ID:COSMIC    17749961 non-null object
VALUE                  17749961 non-null float64
CELL_LINE_ID           17088039 non-null object
dtypes: float64(1), object(3)
memory usage: 677.1+ MB


## Export

In [15]:
# Ensure there are no values, except for in the common cell line ID -- it's not
# yet clear what this means since no other CGDS datasets have null common IDs like 
# this, but perhaps these measurements will still be useful in some way
assert np.all(pd.notnull(d_exp[['GENE_ID:HGNC', 'CELL_LINE_ID:COSMIC', 'VALUE']]))

# Also ensure that all identifiers are only strings, no ints or floats
for c in ['CELL_LINE_ID:COSMIC', 'CELL_LINE_ID', 'GENE_ID:HGNC']:
    assert np.all(d_exp[c].dropna().apply(type) == str), 'Found non-string id for field "{}"'.format(c)

db.save(d_exp, src.GDSC_v2, db.IMPORT, 'gene-expression')

'/Users/eczech/data/research/mgds/import/gdsc_v2_gene-expression.pkl'